# Completar información

### Unificación inicial de JSONs por ciudad
Se cargan todos los ficheros de restaurantes (`restaurantes_{ciudad}.json`) y se combinan en un único dataset de Galicia.  
El resultado se guarda en **CSV y JSON** (`restaurantes_galicia.csv / .json`) para tener una base consolidada.


In [ ]:
import pandas as pd
import json
import glob

# Cargar todos los JSON descargados
all_files = glob.glob("restaurantes_*.json")  # Ej: restaurantes_vigo.json
df_list = []
all_restaurantes = []

for filename in all_files:
    with open(filename, "r", encoding="utf-8") as f:
        data = json.load(f)

        # Normalizar a DataFrame para CSV
        df = pd.json_normalize(data)
        df["ciudad"] = filename.split("_")[1].replace(".json", "")
        df_list.append(df)

        # Mantener lista de dicts para JSON
        for r in data:
            r["ciudad"] = filename.split("_")[1].replace(".json", "")
            all_restaurantes.append(r)

# --- Guardado en CSV ---
full_df = pd.concat(df_list, ignore_index=True)
full_df.to_csv("restaurantes_galicia.csv", index=False, encoding="utf-8")

# --- Guardado en JSON ---
with open("restaurantes_galicia.json", "w", encoding="utf-8") as f:
    json.dump(all_restaurantes, f, ensure_ascii=False, indent=4)

print(f"✅ Dataset consolidado: {len(all_restaurantes)} restaurantes guardados en CSV y JSON")


✅ Dataset consolidado: 3986 restaurantes guardados en CSV y JSON


### Separaciónde restaurantes incompletos
Se separan aquellos restaurantes marcados con el flag `detalle_completo = False`.

In [ ]:
import json

# --- Archivos de entrada y salida ---
INPUT_FILE = "restaurantes_galicia.json"
OUTPUT_INCOMPLETOS = "incompletos.json"
OUTPUT_COMPLETOS = "restaurantes_ok.json"

# --- Cargar los datos ---
with open(INPUT_FILE, "r", encoding="utf-8") as f:
    restaurantes = json.load(f)

# --- Separar los incompletos y los completos ---
incompletos = [r for r in restaurantes if not r.get("detalle_completo", True)]
completos = [r for r in restaurantes if r.get("detalle_completo", True)]

# --- Guardar ambos resultados ---
with open(OUTPUT_INCOMPLETOS, "w", encoding="utf-8") as f:
    json.dump(incompletos, f, ensure_ascii=False, indent=4)

with open(OUTPUT_COMPLETOS, "w", encoding="utf-8") as f:
    json.dump(completos, f, ensure_ascii=False, indent=4)

# --- Mostrar resumen ---
print(f"✅ Archivo '{OUTPUT_INCOMPLETOS}' creado con {len(incompletos)} restaurantes incompletos.")
print(f"✅ Archivo '{OUTPUT_COMPLETOS}' creado con {len(completos)} restaurantes completos.")
print(f"📊 Total procesado: {len(restaurantes)} registros.")


✅ Archivo 'incompletos.json' creado con 1993 restaurantes incompletos.
✅ Archivo 'restaurantes_ok.json' creado con 1993 restaurantes completos.
📊 Total procesado: 3986 registros.


### Enriquecimiento con Google Places (detalles básicos)
Se buscan los restaurantes incompletos en la API de Google Places (Nearby Search + Details).  
Se rellenan **dirección, teléfono, web, horarios, precio, rating y nº reseñas**.  
El resultado se guarda como `restaurantes_completados_con_google.json`.


In [ ]:
import requests
import json
import time

API_KEY = ""  # Sustituye esto por tu clave real
INPUT_FILE = "restaurantes_incompletos.json"
OUTPUT_FILE = "restaurantes_completados_con_google.json"
LANG = "es"
RADIUS = 100  # radio en metros para búsqueda local

def buscar_place_id(nombre, lat, lng):
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        "key": API_KEY,
        "location": f"{lat},{lng}",
        "radius": RADIUS,
        "keyword": nombre,
        "type": "restaurant",
        "language": LANG
    }
    r = requests.get(url, params=params)
    resultados = r.json().get("results", [])
    return resultados[0].get("place_id") if resultados else None

def obtener_detalles_place(place_id):
    url = "https://maps.googleapis.com/maps/api/place/details/json"
    params = {
        "key": API_KEY,
        "place_id": place_id,
        "language": LANG,
        "fields": ",".join([
            "formatted_address",
            "formatted_phone_number",
            "website",
            "opening_hours",
            "price_level",
            "rating",
            "user_ratings_total",
            "reviews"
        ])
    }
    r = requests.get(url, params=params)
    return r.json().get("result", {})

# Cargar restaurantes incompletos
with open(INPUT_FILE, "r", encoding="utf-8") as f:
    restaurantes = json.load(f)

enriquecidos = []

for rest in restaurantes:
    nombre = rest.get("name")
    lat = rest.get("latitude")
    lng = rest.get("longitude")

    print(f"🔎 Buscando: {nombre}")
    place_id = buscar_place_id(nombre, lat, lng)

    if not place_id:
        print("❌ No se encontró el restaurante en Google Places")
        enriquecidos.append(rest)
        continue

    detalles = obtener_detalles_place(place_id)
    print(f"✅ Encontrado: {detalles.get('formatted_address', 'sin dirección')}")

    # Añadir campos si están vacíos
    rest["address"] = detalles.get("formatted_address", rest.get("address"))
    rest["phone"] = detalles.get("formatted_phone_number", rest.get("phone"))
    rest["website"] = detalles.get("website", rest.get("website"))

    horarios = detalles.get("opening_hours", {}).get("weekday_text")
    if horarios:
        rest["open_hours_google"] = horarios  # Campo auxiliar para conservarlo

    rest["rating_google"] = detalles.get("rating")
    rest["reviews_google"] = detalles.get("user_ratings_total")
    rest["price_level_google"] = detalles.get("price_level")

    enriquecidos.append(rest)
    time.sleep(0.5)  # evita sobrepasar límite de peticiones

# Guardar resultado enriquecido
with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    json.dump(enriquecidos, f, ensure_ascii=False, indent=4)

print(f"\n📦 Archivo guardado como '{OUTPUT_FILE}' con {len(enriquecidos)} restaurantes enriquecidos.")


#### Guardado de resultados
Se separan los restaurantes encontrados (`restaurantes_completados_con_google.json`) de los no encontrados (`no_encontrados.json`).


### Estrategias avanzadas de búsqueda
Se refinan los intentos para localizar restaurantes difíciles:  
- Normalización de nombres.  
- Prefijos comunes (*Bar*, *Restaurante*, *Taberna*…).  
- Filtros por coordenadas (radio en metros) a partir del cálculo de distancias (Haversine).
Estos pasos aumentan la tasa de resolución.


In [ ]:
pip install unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 7.2 MB/s eta 0:00:00


In [ ]:
import requests
import json
import time
import difflib
import re
import math
import unidecode

API_KEY = ""  # ← Reemplaza con tu clave
INPUT_FILE = "no_encontrados.json"
OUTPUT_FILE = "restaurantes_completados_apiv2.json"
NO_ENCONTRADOS_FILE = "no_encontrados_15.json"
LANG = "es"
RADIUS_METERS = 400
MAX_LLAMADAS = 50000
contador_llamadas = 0

# ----------------- utilidades -----------------
def normalizar_nombre(nombre):
    if not nombre:
        return ""
    nombre = unidecode.unidecode(nombre.lower())
    nombre = re.sub(r"\b(bar|restaurante|cerveceria|cafeteria|taberna)\b", "", nombre)
    nombre = re.sub(r"\b(o|el|la|los|las|de|del|dos|da|do|das|a|en|y|e)\b", "", nombre)
    nombre = re.sub(r"[^a-z0-9\s]", "", nombre)
    nombre = re.sub(r"\s+", " ", nombre)
    return nombre.strip()

def haversine_m(lat1, lon1, lat2, lon2):
    R = 6371000.0  # metros
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlmb = math.radians(lon2 - lon1)
    a = math.sin(dphi/2)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(dlmb/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    return R * c


def nearby_por_nombre(nombre, lat, lng):
    """Búsqueda estricta por radio usando places:searchNearby y fuzzy por nombre."""
    global contador_llamadas
    url = "https://places.googleapis.com/v1/places:searchNearby"
    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": API_KEY,
        "X-Goog-FieldMask": "places.id,places.displayName,places.location"
    }
    data = {
        "languageCode": LANG,
        "maxResultCount": 20,
        "locationRestriction": {
            "circle": {
                "center": {"latitude": lat, "longitude": lng},
                "radius": RADIUS_METERS
            }
        },
        "rankPreference": "DISTANCE"
    }
    r = requests.post(url, headers=headers, json=data)
    contador_llamadas += 1

    if r.status_code != 200:
        print(f"⚠️ Error nearby: {r.status_code} → {r.text}")
        return None

    candidatos = r.json().get("places", [])
    if not candidatos:
        return None

    # Elegimos el más parecido por nombre
    base = nombre.lower()
    mejores = []
    for p in candidatos:
        nm = (p.get("displayName") or {}).get("text", "")
        sim = difflib.SequenceMatcher(None, base, nm.lower()).ratio()
        mejores.append((sim, p))
    mejores.sort(key=lambda x: x[0], reverse=True)

    # Aceptamos si supera un umbral razonable
    if mejores and mejores[0][0] >= 0.40:
        return mejores[0][1].get("id")
    return None

def searchtext_acotado(nombre, lat=None, lng=None):
    """Búsqueda por texto con locationBias, pero filtrando por distancia <= RADIUS_METERS."""
    global contador_llamadas
    url = "https://places.googleapis.com/v1/places:searchText"
    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": API_KEY,
        # pedimos también location para poder filtrar por distancia
        "X-Goog-FieldMask": "places.id,places.displayName,places.location"
    }
    data = {"textQuery": nombre, "languageCode": LANG}
    if lat is not None and lng is not None:
        data["locationBias"] = {
            "circle": {
                "center": {"latitude": lat, "longitude": lng},
                "radius": RADIUS_METERS
            }
        }
    r = requests.post(url, headers=headers, json=data)
    contador_llamadas += 1

    if r.status_code != 200:
        print(f"⚠️ Error searchText: {r.status_code} → {r.text}")
        return None

    resultados = r.json().get("places", [])
    if not resultados:
        return None

    # Filtramos por distancia estricta si tenemos coords
    if lat is not None and lng is not None:
        dentro = []
        for p in resultados:
            loc = p.get("location")
            if loc:
                d = haversine_m(lat, lng, loc.get("latitude"), loc.get("longitude"))
                if d <= RADIUS_METERS:
                    dentro.append(p)
        if dentro:
            resultados = dentro
        else:
            # si ninguno cae dentro del radio, lo consideramos fallo
            return None

    # Elegimos por similitud de nombre
    base = nombre.lower()
    mejores = []
    for p in resultados:
        nm = (p.get("displayName") or {}).get("text", "")
        sim = difflib.SequenceMatcher(None, base, nm.lower()).ratio()
        mejores.append((sim, p))
    mejores.sort(key=lambda x: x[0], reverse=True)

    if mejores and mejores[0][0] >= 0.40:
        return mejores[0][1].get("id")
    return None

def buscar_place_id(nombre_original, lat=None, lng=None):
    # 1) si hay coordenadas, probamos nearby (radio duro)
    if lat is not None and lng is not None:
        pid = nearby_por_nombre(nombre_original, lat, lng)
        if pid:
            return pid
        # reintento con nombre normalizado
        pid = nearby_por_nombre(normalizar_nombre(nombre_original), lat, lng)
        if pid:
            return pid
        # con prefijos comunes
        for prefijo in ["Bar", "Restaurante", "Cafetería", "Cervecería", "Taberna"]:
            pid = nearby_por_nombre(f"{prefijo} {nombre_original}", lat, lng)
            if pid:
                return pid

    # 2) fallback: searchText con sesgo + filtro de distancia
    pid = searchtext_acotado(nombre_original, lat, lng)
    if pid:
        return pid
    pid = searchtext_acotado(normalizar_nombre(nombre_original), lat, lng)
    if pid:
        return pid
    for prefijo in ["Bar", "Restaurante", "Cafetería", "Cervecería", "Taberna"]:
        pid = searchtext_acotado(f"{prefijo} {nombre_original}", lat, lng)
        if pid:
            return pid

    return None

# ----------------- detalles -----------------
def obtener_detalles_place(place_id):
    global contador_llamadas
    url = f"https://places.googleapis.com/v1/places/{place_id}"
    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": API_KEY,
        "X-Goog-FieldMask": ",".join([
            "displayName",
            "formattedAddress",
            "internationalPhoneNumber",
            "regularOpeningHours",
            "websiteUri",
            "rating",
            "userRatingCount",
            "priceLevel",
            "reservable",
            "delivery"
        ])
    }
    r = requests.get(url, headers=headers)
    contador_llamadas += 1
    if r.status_code != 200:
        print(f"⚠️ Error en detalles: {r.status_code} → {r.text}")
        return {}
    return r.json()

# ----------------- ejecución -----------------
with open(INPUT_FILE, "r", encoding="utf-8") as f:
    restaurantes = json.load(f)

enriquecidos = []
no_encontrados = []

for idx, rest in enumerate(restaurantes):
    if contador_llamadas >= MAX_LLAMADAS:
        print("⚠️ Límite de llamadas alcanzado. Deteniendo proceso.")
        break

    nombre_original = rest.get("name")
    lat = rest.get("latitude")
    lng = rest.get("longitude")

    print(f"[{idx+1}/{len(restaurantes)}] 🔎 Buscando: {nombre_original}")
    place_id = buscar_place_id(nombre_original, lat, lng)

    if not place_id:
        print("❌ No encontrado en Google Places")
        no_encontrados.append(rest)
        continue

    detalles = obtener_detalles_place(place_id)
    print(f"✅ Encontrado: {detalles.get('formattedAddress', 'sin dirección')}")

    rest["address"] = detalles.get("formattedAddress", rest.get("address"))
    rest["phone"] = detalles.get("internationalPhoneNumber", rest.get("phone"))
    rest["website"] = detalles.get("websiteUri", rest.get("website"))
    rest["rating_google"] = detalles.get("rating")
    rest["reviews_count_google"] = detalles.get("userRatingCount")
    rest["price_level_google"] = detalles.get("priceLevel")
    rest["reservable"] = detalles.get("reservable")
    rest["delivery"] = detalles.get("delivery")

    display_name = (detalles.get("displayName") or {}).get("text")
    if display_name:
        rest["name_google"] = display_name

    horarios = (detalles.get("regularOpeningHours") or {}).get("weekdayDescriptions")
    if horarios:
        rest["open_hours_google"] = horarios

    enriquecidos.append(rest)
    time.sleep(0.5)

    if (idx + 1) % 100 == 0:
        print(f"📊 Progreso: {idx+1} procesados — {contador_llamadas} llamadas usadas")

with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    json.dump(enriquecidos, f, ensure_ascii=False, indent=4)

with open(NO_ENCONTRADOS_FILE, "w", encoding="utf-8") as f:
    json.dump(no_encontrados, f, ensure_ascii=False, indent=4)

print(f"\n✅ Archivo guardado como '{OUTPUT_FILE}' con {len(enriquecidos)} restaurantes.")
print(f"❗ Guardado archivo '{NO_ENCONTRADOS_FILE}' con {len(no_encontrados)} no encontrados.")



[1/1184] 🔎 Buscando: Bar Las cinco vigas
✅ Encontrado: Rúa da Cruz, 5, 27001 Lugo, Spain
[2/1184] 🔎 Buscando: Casa de aira padrón
✅ Encontrado: Rúa Fontao, 15, 27620 Samos, Lugo, Spain
[3/1184] 🔎 Buscando: Cafetería Centro Deportivo D10
❌ No encontrado en Google Places
[4/1184] 🔎 Buscando: Maty's Café
✅ Encontrado: Pr. Maior, 30, 27001 Lugo, Spain
[5/1184] 🔎 Buscando: Restaurante Terras do Miño
✅ Encontrado: 27002 Lugo, Spain
[6/1184] 🔎 Buscando: El Sagrado Taquería
❌ No encontrado en Google Places
[7/1184] 🔎 Buscando: Bodegas Do San Vicente
✅ Encontrado: Rúa Nova, 5, 27001 Lugo, Spain
[8/1184] 🔎 Buscando: Meson Parrillada A Cova
✅ Encontrado: 27548 Os Ferreiros, Lugo, Spain
[9/1184] 🔎 Buscando: O Sacho
✅ Encontrado: Estrada da Granxa, 29, 27002 Lugo, Spain
[10/1184] 🔎 Buscando: Restaurante Los Robles
✅ Encontrado: Calle San Miguel Orbazay Islas, 0 S-N, 27001 Lugo, Spain
[11/1184] 🔎 Buscando: Parrillada Hombreiro
✅ Encontrado: Rúa Santiago, 216, 27004 Lugo, Spain
[12/1184] 🔎 Buscando: 

### Verificación de coordenadas
Se añaden filtros por **bounding box de Galicia** para tratar de encontrar los restantes.  
Con esto se descartan resultados fuera de la comunidad o con coordenadas sospechosas.


In [ ]:
import requests
import json
import time
import difflib
import re
import math
import unidecode

API_KEY = ""  # ← Reemplaza con tu clave
INPUT_FILE = "no_encontrados_15.json"
OUTPUT_FILE = "restaurantes_completados_15.json"
NO_ENCONTRADOS_FILE = "no_encontrados_15_.json"
LANG = "es"
RADIUS_METERS = 400
MAX_LLAMADAS = 50000
contador_llamadas = 0

# Bounding box aproximado de Galicia
BOUNDING_BOX_GALICIA = {
    "min_lat": 41.80,
    "max_lat": 43.85,
    "min_lng": -9.35,
    "max_lng": -6.50
}

PROVINCIAS_GALICIA = [
    "A Coruña", "A Coruna", "Coruña", "Coruna",
    "Lugo",
    "Ourense", "Orense",
    "Pontevedra",
    "Galicia"
]

# ----------------- utilidades -----------------
def normalizar_nombre(nombre):
    if not nombre:
        return ""
    nombre = unidecode.unidecode(nombre.lower())
    nombre = re.sub(r"\b(bar|restaurante|cerveceria|cafeteria|taberna)\b", "", nombre)
    nombre = re.sub(r"\b(o|el|la|los|las|de|del|dos|da|do|das|a|en|y|e)\b", "", nombre)
    nombre = re.sub(r"[^a-z0-9\s]", "", nombre)
    nombre = re.sub(r"\s+", " ", nombre)
    return nombre.strip()

def haversine_m(lat1, lon1, lat2, lon2):
    R = 6371000.0  # metros
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlmb = math.radians(lon2 - lon1)
    a = math.sin(dphi/2)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(dlmb/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    return R * c

def en_bbox_galicia(lat, lng):
    if lat is None or lng is None:
        return False
    return (BOUNDING_BOX_GALICIA["min_lat"] <= lat <= BOUNDING_BOX_GALICIA["max_lat"] and
            BOUNDING_BOX_GALICIA["min_lng"] <= lng <= BOUNDING_BOX_GALICIA["max_lng"])

def texto_parece_galicia(address):
    if not address:
        return False
    a = unidecode.unidecode(address).lower()
    for prov in PROVINCIAS_GALICIA:
        if unidecode.unidecode(prov).lower() in a:
            return True
    return False

def nearby_por_nombre(nombre, lat, lng):
    """Búsqueda estricta por radio usando places:searchNearby y fuzzy por nombre."""
    global contador_llamadas
    url = "https://places.googleapis.com/v1/places:searchNearby"
    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": API_KEY,
        "X-Goog-FieldMask": "places.id,places.displayName,places.location,places.formattedAddress"
    }
    data = {
        "languageCode": LANG,
        "maxResultCount": 20,
        "locationRestriction": {
            "circle": {
                "center": {"latitude": lat, "longitude": lng},
                "radius": RADIUS_METERS
            }
        },
        "rankPreference": "DISTANCE"
    }
    r = requests.post(url, headers=headers, json=data)
    contador_llamadas += 1

    if r.status_code != 200:
        print(f"⚠️ Error nearby: {r.status_code} → {r.text}")
        return None

    candidatos = r.json().get("places", [])
    if not candidatos:
        return None

    # Elegimos el más parecido por nombre
    base = (nombre or "").lower()
    mejores = []
    for p in candidatos:
        nm = (p.get("displayName") or {}).get("text", "")
        sim = difflib.SequenceMatcher(None, base, nm.lower()).ratio()
        mejores.append((sim, p))
    mejores.sort(key=lambda x: x[0], reverse=True)

    # Aceptamos si supera un umbral razonable
    if mejores and mejores[0][0] >= 0.40:
        return mejores[0][1].get("id")
    return None

def searchtext_contraido(nombre, lat=None, lng=None):
    """
    Búsqueda por texto.
    - Si hay coords: usa locationBias circle y filtra por distancia <= RADIUS_METERS.
    - Si NO hay coords: limita a Galicia con locationBias rectangle y filtra por bbox/provincia.
    """
    global contador_llamadas
    url = "https://places.googleapis.com/v1/places:searchText"
    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": API_KEY,
        # pedimos location y address para filtrar
        "X-Goog-FieldMask": "places.id,places.displayName,places.location,places.formattedAddress"
    }

    data = {
        "textQuery": nombre,
        "languageCode": LANG,
        "regionCode": "ES"
    }

    if lat is not None and lng is not None:
        # Caso con coordenadas: sesgo circular y filtro estricto por distancia
        data["locationBias"] = {
            "circle": {
                "center": {"latitude": lat, "longitude": lng},
                "radius": RADIUS_METERS
            }
        }
    else:
        # Sin coordenadas: sesgo rectangular dentro de Galicia
        data["locationBias"] = {
            "rectangle": {
                "low": {
                    "latitude": BOUNDING_BOX_GALICIA["min_lat"],
                    "longitude": BOUNDING_BOX_GALICIA["min_lng"]
                },
                "high": {
                    "latitude": BOUNDING_BOX_GALICIA["max_lat"],
                    "longitude": BOUNDING_BOX_GALICIA["max_lng"]
                }
            }
        }

    r = requests.post(url, headers=headers, json=data)
    contador_llamadas += 1

    if r.status_code != 200:
        print(f"⚠️ Error searchText: {r.status_code} → {r.text}")
        return None

    resultados = r.json().get("places", [])
    if not resultados:
        return None

    # Filtrado
    filtrados = []
    if lat is not None and lng is not None:
        # Filtramos por distancia estricta
        for p in resultados:
            loc = p.get("location")
            if loc:
                d = haversine_m(lat, lng, loc.get("latitude"), loc.get("longitude"))
                if d <= RADIUS_METERS:
                    filtrados.append(p)
    else:
        # Sin coords: quedarnos sólo con resultados dentro del bbox Galicia
        for p in resultados:
            loc = p.get("location") or {}
            plat = loc.get("latitude")
            plng = loc.get("longitude")
            addr = p.get("formattedAddress")
            if en_bbox_galicia(plat, plng) or texto_parece_galicia(addr):
                filtrados.append(p)

    if not filtrados:
        return None

    # Elegimos por similitud de nombre
    base = (nombre or "").lower()
    mejores = []
    for p in filtrados:
        nm = (p.get("displayName") or {}).get("text", "")
        sim = difflib.SequenceMatcher(None, base, nm.lower()).ratio()
        mejores.append((sim, p))
    mejores.sort(key=lambda x: x[0], reverse=True)

    if mejores and mejores[0][0] >= 0.40:
        return mejores[0][1].get("id")
    return None

def buscar_place_id(nombre_original, lat=None, lng=None):
    # 1) si hay coordenadas, probamos nearby (radio duro)
    if lat is not None and lng is not None:
        pid = nearby_por_nombre(nombre_original, lat, lng)
        if pid:
            return pid
        # reintento con nombre normalizado
        pid = nearby_por_nombre(normalizar_nombre(nombre_original), lat, lng)
        if pid:
            return pid
        # con prefijos comunes
        for prefijo in ["Bar", "Restaurante", "Cafetería", "Cervecería", "Taberna", "Cafeteria", "Cerveceria"]:
            pid = nearby_por_nombre(f"{prefijo} {nombre_original}", lat, lng)
            if pid:
                return pid

    # 2) fallback: searchText con sesgo y filtros (radio si hay coords, Galicia si no hay)
    pid = searchtext_contraido(nombre_original, lat, lng)
    if pid:
        return pid
    pid = searchtext_contraido(normalizar_nombre(nombre_original), lat, lng)
    if pid:
        return pid
    for prefijo in ["Bar", "Restaurante", "Cafetería", "Cervecería", "Taberna", "Cafeteria", "Cerveceria"]:
        pid = searchtext_contraido(f"{prefijo} {nombre_original}", lat, lng)
        if pid:
            return pid

    return None

# ----------------- detalles -----------------
def obtener_detalles_place(place_id):
    global contador_llamadas
    url = f"https://places.googleapis.com/v1/places/{place_id}"
    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": API_KEY,
        "X-Goog-FieldMask": ",".join([
            "displayName",
            "formattedAddress",
            "internationalPhoneNumber",
            "regularOpeningHours",
            "websiteUri",
            "rating",
            "userRatingCount",
            "priceLevel",
            "reservable",
            "delivery"
        ])
    }
    r = requests.get(url, headers=headers)
    contador_llamadas += 1
    if r.status_code != 200:
        print(f"⚠️ Error en detalles: {r.status_code} → {r.text}")
        return {}
    return r.json()

# ----------------- ejecución -----------------
with open(INPUT_FILE, "r", encoding="utf-8") as f:
    restaurantes = json.load(f)

enriquecidos = []
no_encontrados = []

for idx, rest in enumerate(restaurantes):
    if contador_llamadas >= MAX_LLAMADAS:
        print("⚠️ Límite de llamadas alcanzado. Deteniendo proceso.")
        break

    nombre_original = rest.get("name")
    lat = rest.get("latitude")
    lng = rest.get("longitude")

    print(f"[{idx+1}/{len(restaurantes)}] 🔎 Buscando: {nombre_original}")

    # Si NO hay coordenadas, y por seguridad vienen fuera de Galicia, descártalo
    if (lat is not None and lng is not None) and not en_bbox_galicia(lat, lng):
        print("🚫 Coordenadas fuera de Galicia; se intentará sólo por nombre dentro de Galicia.")
        lat = None
        lng = None

    place_id = buscar_place_id(nombre_original, lat, lng)

    if not place_id:
        print("❌ No encontrado en Google Places (limitado a Galicia)")
        no_encontrados.append(rest)
        continue

    detalles = obtener_detalles_place(place_id)
    print(f"✅ Encontrado: {detalles.get('formattedAddress', 'sin dirección')}")

    rest["address"] = detalles.get("formattedAddress", rest.get("address"))
    rest["phone"] = detalles.get("internationalPhoneNumber", rest.get("phone"))
    rest["website"] = detalles.get("websiteUri", rest.get("website"))
    rest["rating_google"] = detalles.get("rating")
    rest["reviews_count_google"] = detalles.get("userRatingCount")
    rest["price_level_google"] = detalles.get("priceLevel")
    rest["reservable"] = detalles.get("reservable")
    rest["delivery"] = detalles.get("delivery")

    display_name = (detalles.get("displayName") or {}).get("text")
    if display_name:
        rest["name_google"] = display_name

    horarios = (detalles.get("regularOpeningHours") or {}).get("weekdayDescriptions")
    if horarios:
        rest["open_hours_google"] = horarios

    enriquecidos.append(rest)
    time.sleep(0.5)

    if (idx + 1) % 100 == 0:
        print(f"📊 Progreso: {idx+1} procesados — {contador_llamadas} llamadas usadas")

with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    json.dump(enriquecidos, f, ensure_ascii=False, indent=4)

with open(NO_ENCONTRADOS_FILE, "w", encoding="utf-8") as f:
    json.dump(no_encontrados, f, ensure_ascii=False, indent=4)

print(f"\n✅ Archivo guardado como '{OUTPUT_FILE}' con {len(enriquecidos)} restaurantes.")
print(f"❗ Guardado archivo '{NO_ENCONTRADOS_FILE}' con {len(no_encontrados)} no encontrados.")


[1/17] 🔎 Buscando: Cafetería Centro Deportivo D10
❌ No encontrado en Google Places (limitado a Galicia)
[2/17] 🔎 Buscando: Cafetería Estación de Servicio O Corgo
❌ No encontrado en Google Places (limitado a Galicia)
[3/17] 🔎 Buscando: Cafetería Ponte Branca
❌ No encontrado en Google Places (limitado a Galicia)
[4/17] 🔎 Buscando: cerveceria la penultima
✅ Encontrado: Polígono Industrial do Ceao, Avenida Benigno Rivera, Local 15, 27003 Lugo, Spain
[5/17] 🔎 Buscando: Restaurante los Robles
❌ No encontrado en Google Places (limitado a Galicia)
[6/17] 🔎 Buscando: Restaurante Tarará
❌ No encontrado en Google Places (limitado a Galicia)
[7/17] 🔎 Buscando: La Cervecería Internacional
❌ No encontrado en Google Places (limitado a Galicia)
[8/17] 🔎 Buscando: Parador la Rocha
❌ No encontrado en Google Places (limitado a Galicia)
[9/17] 🔎 Buscando: Bar Agacha A Testa
❌ No encontrado en Google Places (limitado a Galicia)
[10/17] 🔎 Buscando: Cafeteria HOLLYWOOD
✅ Encontrado: Av. de Castrelos, 166, Fr

In [ ]:
import json

# --- Archivos de entrada ---
archivos = [
    "restaurantes_ok.json",
    "restaurantes_completados_con_google.json",
    "restaurantes_completados_apiv2.json",
    "restaurantes_completados_15.json"
]

# --- Archivo de salida ---
OUTPUT_FILE = "restaurantes_añadir_url.json"

# --- Cargar y unificar ---
todos = {}

for archivo in archivos:
    try:
        with open(archivo, "r", encoding="utf-8") as f:
            datos = json.load(f)
            print(f"📥 Cargando {archivo}: {len(datos)} registros")
            for r in datos:
                rid = r.get("id")
                if rid is not None:
                    todos[rid] = r  # mantiene el último si hay duplicados
    except FileNotFoundError:
        print(f"⚠️ Archivo no encontrado: {archivo}")

# --- Convertir a lista ---
resultado = list(todos.values())

# --- Guardar ---
with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    json.dump(resultado, f, ensure_ascii=False, indent=4)

print(f"\n✅ Archivo '{OUTPUT_FILE}' creado con {len(resultado)} restaurantes unificados.")


###  Añadir URL oficial de Google Maps
Se guarda el enlace directo a Google Maps (`google_maps_url`).


In [ ]:
import requests
import json
import time
import difflib
import re
import unidecode

# ======= CONFIG =======
API_KEY = ""  # ← pon tu clave
INPUT_FILE = "restaurantes_añadir_url.json"
OUTPUT_FILE = "restaurantes_con_url.json"
NO_ENCONTRADOS_FILE = "no_encontrados_url.json"

LANG = "es"
MAX_LLAMADAS = 50000
contador_llamadas = 0

# Límite de consultas de Text Search por sitio (cap blando por pasada)
MAX_QUERIES_PER_SITE = 6

# Bounding box aproximado de Galicia
BOUNDING_BOX_GALICIA = {
    "min_lat": 41.80,
    "max_lat": 43.85,
    "min_lng": -9.35,
    "max_lng": -6.50
}

PROVINCIAS_GALICIA = [
    "A Coruña", "A Coruna", "Coruña", "Coruna",
    "Lugo",
    "Ourense", "Orense",
    "Pontevedra",
    "Galicia"
]

# Umbrales de similitud (nombre)
THRESHOLD_SIM_FUERTE = 0.86
THRESHOLD_SIM_MINIMO = 0.40

# Peso combinado nombre + dirección cuando usamos address como apoyo
WEIGHT_NAME = 0.7
WEIGHT_ADDR = 0.3

# ======= UTILIDADES =======
STOPWORDS_NOMBRES = {
    "bar","restaurante","cerveceria","cafeteria","taberna",
    "o","el","la","los","las","de","del","dos","da","do","das","a","en","y","e"
}
STOPWORDS_DIRECCION = {
    "rua","r","calle","avenida","av","estrada","praza","plaza","travesia","carretera",
    "n","lg","lugar","parroquia","galicia","spain","españa","a","de","del","da","do","das",
    "provincia","concello","municipio","lugo","coruna","a coruna","ourense","orense","pontevedra"
}

def normalizar_texto(s: str) -> str:
    if not s:
        return ""
    s = unidecode.unidecode(s.lower())
    s = re.sub(r"[^a-z0-9\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def normalizar_nombre(nombre: str) -> str:
    s = normalizar_texto(nombre)
    tokens = [t for t in s.split() if t not in STOPWORDS_NOMBRES]
    return " ".join(tokens)

def normalizar_direccion(addr: str) -> str:
    s = normalizar_texto(addr)
    return s

def tokens_filtrados(s: str, stopwords: set) -> set:
    return set(t for t in normalizar_texto(s).split() if t and t not in stopwords)

def sim(a: str, b: str) -> float:
    return difflib.SequenceMatcher(None, (a or ""), (b or "")).ratio()

def en_bbox_galicia(lat, lng) -> bool:
    if lat is None or lng is None:
        return False
    return (BOUNDING_BOX_GALICIA["min_lat"] <= lat <= BOUNDING_BOX_GALICIA["max_lat"] and
            BOUNDING_BOX_GALICIA["min_lng"] <= lng <= BOUNDING_BOX_GALICIA["max_lng"])

def texto_parece_galicia(address: str) -> bool:
    if not address:
        return False
    a = unidecode.unidecode(address).lower()
    for prov in PROVINCIAS_GALICIA:
        if unidecode.unidecode(prov).lower() in a:
            return True
    return False

def extraer_localidad_de_address(addr: str) -> str:
    """
    Heurística simple: usamos la penúltima o antepenúltima parte de 'a, b, c, d' como localidad.
    """
    if not addr:
        return ""
    parts = [p.strip() for p in addr.split(",") if p.strip()]
    if len(parts) >= 3:
        return parts[-3]  # p.ej.: "Rúa X", "CP Ciudad", "Provincia", "País" → ciudad ≈ -3
    if len(parts) >= 2:
        return parts[-2]
    return parts[-1] if parts else ""

# ======= GOOGLE PLACES =======
def search_text_galicia(query: str):
    """Busca por texto con sesgo rectangular en Galicia y devuelve lista de places con id, displayName, location y address."""
    global contador_llamadas
    url = "https://places.googleapis.com/v1/places:searchText"
    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": API_KEY,
        "X-Goog-FieldMask": "places.id,places.displayName,places.location,places.formattedAddress"
    }
    data = {
        "textQuery": query,
        "languageCode": LANG,
        "regionCode": "ES",
        "locationBias": {
            "rectangle": {
                "low": {
                    "latitude": BOUNDING_BOX_GALICIA["min_lat"],
                    "longitude": BOUNDING_BOX_GALICIA["min_lng"]
                },
                "high": {
                    "latitude": BOUNDING_BOX_GALICIA["max_lat"],
                    "longitude": BOUNDING_BOX_GALICIA["max_lng"]
                }
            }
        }
    }
    r = requests.post(url, headers=headers, json=data)
    contador_llamadas += 1

    if r.status_code != 200:
        print(f"⚠️ Error searchText: {r.status_code} → {r.text}")
        return []

    resultados = r.json().get("places", []) or []
    # Filtramos a Galicia por bbox o mención en la dirección
    filtrados = []
    for p in resultados:
        loc = p.get("location") or {}
        plat = loc.get("latitude")
        plng = loc.get("longitude")
        addr = p.get("formattedAddress")
        if en_bbox_galicia(plat, plng) or texto_parece_galicia(addr):
            filtrados.append(p)
    return filtrados

def obtener_detalles_place(place_id):
    """Recupera detalles adicionales + enlaces y flags (incluye location para poder actualizar coords)."""
    global contador_llamadas
    url = f"https://places.googleapis.com/v1/places/{place_id}"
    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": API_KEY,
        "X-Goog-FieldMask": ",".join([
            "displayName",
            "formattedAddress",
            "internationalPhoneNumber",
            "regularOpeningHours",
            "websiteUri",
            "googleMapsUri",
            "rating",
            "userRatingCount",
            "priceLevel",
            "reservable",
            "delivery",
            "location"
        ])
    }
    r = requests.get(url, headers=headers)
    contador_llamadas += 1
    if r.status_code != 200:
        print(f"⚠️ Error en detalles: {r.status_code} → {r.text}")
        return {}
    return r.json()

# ======= MATCHING =======
def elegir_mejor_candidato(nombre, candidatos, addr_usuario: str = None):
    """
    Devuelve (place, tipo_match) donde tipo_match ∈ {"perfecto","fuerte","debil", None}
    - Se compara SIEMPRE contra 'name' (no usamos name_google para decidir).
    - Si addr_usuario está presente, ponderamos una puntuación combinada nombre+dirección.
    """
    n1 = normalizar_nombre(nombre or "")

    addr_norm_user = normalizar_direccion(addr_usuario or "")
    addr_tokens_user = tokens_filtrados(addr_norm_user, STOPWORDS_DIRECCION) if addr_norm_user else set()

    mejor = None
    mejor_tipo = None
    mejor_score = -1.0

    for p in candidatos:
        dn = (p.get("displayName") or {}).get("text", "")
        dn_norm = normalizar_nombre(dn)

        # Perfecto: igualdad exacta tras normalizar nombre
        if dn_norm and dn_norm == n1:
            return p, "perfecto"

        # Similitud de nombre
        s_name = sim(dn_norm, n1)

        # Similitud de dirección (si aplica)
        s_addr = 0.0
        addr_g = (p.get("formattedAddress") or "")
        if addr_norm_user and addr_g:
            addr_norm_g = normalizar_direccion(addr_g)
            s_addr = sim(addr_norm_g, addr_norm_user)

            # refuerzo por tokens compartidos en dirección (calle, municipio, etc.)
            user_tokens = addr_tokens_user
            g_tokens = tokens_filtrados(addr_norm_g, STOPWORDS_DIRECCION)
            overlap = len(user_tokens & g_tokens)

            # Si hay poca similitud de nombre, exigimos al menos algo de solape en dirección
            if s_name < THRESHOLD_SIM_FUERTE and overlap < 2:
                # penalizamos score para evitar falsos positivos
                s_addr *= 0.6

        # Score combinado si hay dirección de usuario, si no solo nombre
        score = WEIGHT_NAME * s_name + (WEIGHT_ADDR * s_addr if addr_norm_user else 0.0)

        # Clasificación
        if s_name >= THRESHOLD_SIM_FUERTE:
            tipo = "fuerte"
        elif s_name >= THRESHOLD_SIM_MINIMO:
            # si llega aquí con address y buen s_addr, lo marcamos como "débil" pero podría ganar en score
            tipo = "debil"
        else:
            tipo = None

        if tipo and score > mejor_score:
            mejor = p
            mejor_tipo = tipo
            mejor_score = score

    return mejor, mejor_tipo

# ======= EJECUCIÓN =======
with open(INPUT_FILE, "r", encoding="utf-8") as f:
    restaurantes = json.load(f)

enriquecidos = []
no_encontrados = []

for idx, rest in enumerate(restaurantes):
    if contador_llamadas >= MAX_LLAMADAS:
        print("⚠️ Límite de llamadas alcanzado. Deteniendo proceso.")
        break

    nombre_original = (rest.get("name") or "").strip()

    # Dirección de apoyo (si hay): usa 'address' o 'street'
    addr_usuario = (rest.get("address") or rest.get("street") or "").strip()
    localidad_hint = extraer_localidad_de_address(addr_usuario)

    base_norm = normalizar_nombre(nombre_original)
    print(f"[{idx+1}/{len(restaurantes)}] 🔎 Buscando por nombre: '{nombre_original}'"
          + (f" | addr hint: '{addr_usuario}'" if addr_usuario else ""))

    # ========== PASADA 1: SOLO NOMBRE, CON EARLY STOP PARA MATCH PERFECTO ==========
    queries_1 = []
    if nombre_original:
        queries_1.append(nombre_original)
    if base_norm and base_norm != nombre_original:
        queries_1.append(base_norm)

    prefijos = ["Bar", "Restaurante", "Cafetería", "Cervecería", "Taberna", "Cafeteria", "Cerveceria"]
    for pref in prefijos:
        if nombre_original:
            q = f"{pref} {nombre_original}"
            if q not in queries_1:
                queries_1.append(q)

    if MAX_QUERIES_PER_SITE and len(queries_1) > MAX_QUERIES_PER_SITE:
        queries_1 = queries_1[:MAX_QUERIES_PER_SITE]

    candidatos_por_id = {}
    match_perfecto_encontrado = None

    for q in queries_1:
        if not q:
            continue
        cand = search_text_galicia(q)

        for p in cand:
            pid = p.get("id")
            if pid and pid not in candidatos_por_id:
                candidatos_por_id[pid] = p

        # EARLY STOP: ¿ya hay un match perfecto directo con 'name'?
        for p in cand:
            dn = (p.get("displayName") or {}).get("text", "")
            dn_norm = normalizar_nombre(dn)
            if dn_norm == base_norm:
                match_perfecto_encontrado = p
                break

        if match_perfecto_encontrado:
            break

        time.sleep(0.2)

    # Si ya hay perfecto → detallar y salir
    if match_perfecto_encontrado:
        mejor = match_perfecto_encontrado
        tipo_match = "perfecto"
    else:
        # ========== PASADA 2: APOYO EN DIRECCIÓN (address/street) ==========
        queries_2 = []
        if addr_usuario:
            # 1) dirección tal cual
            queries_2.append(addr_usuario)
            # 2) nombre + localidad
            if localidad_hint:
                queries_2.append(f"{nombre_original} {localidad_hint}")
            # 3) nombre + parte de la dirección (hasta la primera coma)
            primera_linea = addr_usuario.split(",")[0].strip()
            if primera_linea and primera_linea.lower() not in {nombre_original.lower()}:
                queries_2.append(f"{nombre_original} {primera_linea}")
            # 4) calle + localidad
            if primera_linea and localidad_hint:
                queries_2.append(f"{primera_linea} {localidad_hint}")
            # 5) versión normalizada de dirección
            addr_norm = normalizar_direccion(addr_usuario)
            if addr_norm and addr_norm not in queries_2:
                queries_2.append(addr_norm)

        # cap y ejecución
        if MAX_QUERIES_PER_SITE and len(queries_2) > MAX_QUERIES_PER_SITE:
            queries_2 = queries_2[:MAX_QUERIES_PER_SITE]

        for q in queries_2:
            cand = search_text_galicia(q)
            for p in cand:
                pid = p.get("id")
                if pid and pid not in candidatos_por_id:
                    candidatos_por_id[pid] = p
            time.sleep(0.2)

        candidatos = list(candidatos_por_id.values())
        if not candidatos:
            print("❌ No se encontraron candidatos en Galicia.")
            no_encontrados.append(rest)
            continue

        # Elegimos mejor candidato usando nombre + dirección (si la hay)
        mejor, tipo_match = elegir_mejor_candidato(nombre_original, candidatos, addr_usuario)
        if not mejor:
            print("❌ Sin candidato con similitud suficiente.")
            no_encontrados.append(rest)
            continue

    # ========== DETALLES ==========
    place_id = mejor.get("id")
    detalles = obtener_detalles_place(place_id) if place_id else {}
    display_name = (detalles.get("displayName") or {}).get("text") or (mejor.get("displayName") or {}).get("text")
    direccion = detalles.get("formattedAddress") or mejor.get("formattedAddress", "sin dirección")

    print(f"✅ Match {tipo_match}: {display_name} — {direccion}")

    # Enriquecer (no sustituimos 'name'; 'name_google' = oficial de Google)
    rest["name_google"] = display_name or rest.get("name_google")
    rest["address"] = detalles.get("formattedAddress", rest.get("address"))
    rest["phone"] = detalles.get("internationalPhoneNumber", rest.get("phone"))
    rest["website"] = detalles.get("websiteUri", rest.get("website"))
    rest["google_maps_url"] = detalles.get("googleMapsUri")
    rest["rating_google"] = detalles.get("rating")
    rest["reviews_count_google"] = detalles.get("userRatingCount")
    rest["price_level_google"] = detalles.get("priceLevel")
    rest["reservable"] = detalles.get("reservable")
    rest["delivery"] = detalles.get("delivery")

    # Actualizar coordenadas SOLO si el match es perfecto con 'name'
    if tipo_match == "perfecto":
        loc = (detalles.get("location") or mejor.get("location") or {})
        if loc.get("latitude") is not None and loc.get("longitude") is not None:
            rest["latitude"] = loc["latitude"]
            rest["longitude"] = loc["longitude"]

    enriquecidos.append(rest)

    if (idx + 1) % 100 == 0:
        print(f"📊 Progreso: {idx+1} procesados — {contador_llamadas} llamadas usadas")

# ======= SALIDA =======
with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    json.dump(enriquecidos, f, ensure_ascii=False, indent=4)

with open(NO_ENCONTRADOS_FILE, "w", encoding="utf-8") as f:
    json.dump(no_encontrados, f, ensure_ascii=False, indent=4)

print(f"\n✅ Archivo guardado como '{OUTPUT_FILE}' con {len(enriquecidos)} restaurantes.")
print(f"❗ Guardado archivo '{NO_ENCONTRADOS_FILE}' con {len(no_encontrados)} no encontrados.")
print(f"ℹ️ Llamadas usadas: {contador_llamadas}")


[1/740] 🔎 Buscando por nombre: 'Route 66' | addr hint: '32960 Ourense, Province of Ourense, Spain'
✅ Match perfecto: Route 66 — Rúa Alcalde Usero, 7, 15403 Ferrol, A Coruña, Spain
[2/740] 🔎 Buscando por nombre: 'cerveceria la penultima' | addr hint: 'Polígono Industrial do Ceao, Avenida Benigno Rivera, Local 15, 27003 Lugo, Spain'
✅ Match perfecto: Cervecería A Penultima — Rúa Beiramas, 6, 27890 San Cibrao, Lugo, Spain
[3/740] 🔎 Buscando por nombre: 'Cafeteria HOLLYWOOD' | addr hint: 'Av. de Castrelos, 166, Freijeiro, 36210 Vigo, Pontevedra, Spain'
✅ Match perfecto: Hollywood — Rúa Ramón y Cajal, 34, 15006 A Coruña, Spain
[4/740] 🔎 Buscando por nombre: 'Taqueria Masketacos' | addr hint: 'Rúa de Pi y Margall, 145, 36202 Vigo, Pontevedra, Spain'
✅ Match fuerte: Taqueria Masktacos — Rúa de Pi y Margall, 145, 36202 Vigo, Pontevedra, Spain
[5/740] 🔎 Buscando por nombre: 'Takeo Tex-Mex Food&Drinks (A Coruña)' | addr hint: 'Rúa Joaquín Yáñez, 12, 36202 Vigo, Pontevedra, Spain'
✅ Match perfect

KeyboardInterrupt: 

### Completar restaurantes sin horario
En este bloque se procesan aquellos restaurantes que no tenían horarios en el dataset inicial.  
- Se usan las coordenadas y el nombre para buscarlos en la **API de Google Places** (con `searchNearby` o `searchText`).  
- Una vez localizado el restaurante correcto, se recuperan sus detalles (`detalles_place`) incluyendo **horarios, dirección, rating, número de reseñas y precio**.  
- Si el match es **perfecto** se actualizan también las coordenadas.  
- Se generan dos salidas:  
  - `restaurantes_combinados_actualizado_horarios.json` con los restaurantes completados.  
  - `restaurantes_no_encontrados.json` con los que no fue posible localizar.  


In [ ]:
import requests
import json
import time
import difflib
import re
import unidecode
import pandas as pd
from urllib.parse import urlparse

# ======= CONFIG =======
API_KEY = ""   # API key
INPUT_FILE = "restaurantes_con_url.json"
OUTPUT_FILE = "restaurantes_combinados_actualizado_horarios.json"
NO_ENCONTRADOS_FILE = "restaurantes_no_encontrados.json"

LANG = "es"
MAX_LLAMADAS = 50000
contador_llamadas = 0

# Bounding box Galicia
BOUNDING_BOX_GALICIA = {
    "min_lat": 41.80,
    "max_lat": 43.85,
    "min_lng": -9.35,
    "max_lng": -6.50
}

# Umbrales similitud
THRESHOLD_SIM_FUERTE = 0.86
THRESHOLD_SIM_MINIMO = 0.40

# ======= UTILIDADES =======
STOPWORDS_NOMBRES = {
    "bar","restaurante","meson","mesón","parrillada","asador","taperia","tapería","pulperia","pulpería",
    "vinoteca","taberna","cerveceria","cervecería","cafeteria","cafetería","churrasqueria","churrasquería",
    "casa","de","comidas","o","el","la","los","las","del","dos","da","do","das","a","en","y","e"
}

PROVINCIAS_GALICIA = ["lugo","ourense","orense","pontevedra","a coruna","coruna","a coruña","coruña"]

def normalizar_texto(s: str) -> str:
    if not s:
        return ""
    s = unidecode.unidecode(s.lower())
    s = re.sub(r"[^a-z0-9\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def normalizar_nombre(nombre: str) -> str:
    s = normalizar_texto(nombre)
    tokens = [t for t in s.split() if t not in STOPWORDS_NOMBRES]
    return " ".join(tokens)

def normalizar_ciudad(c: str) -> str:
    return unidecode.unidecode((c or "").lower()).strip()

def sim(a: str, b: str) -> float:
    return difflib.SequenceMatcher(None, (a or ""), (b or "")).ratio()

def en_bbox_galicia(lat, lng) -> bool:
    if lat is None or lng is None:
        return False
    return (BOUNDING_BOX_GALICIA["min_lat"] <= lat <= BOUNDING_BOX_GALICIA["max_lat"] and
            BOUNDING_BOX_GALICIA["min_lng"] <= lng <= BOUNDING_BOX_GALICIA["max_lng"])

def direccion_valida(addr: str) -> bool:
    if not addr:
        return False
    addr_norm = unidecode.unidecode(addr.lower())
    if "spain" not in addr_norm and "espana" not in addr_norm:
        return False
    return any(prov in addr_norm for prov in PROVINCIAS_GALICIA)

def direccion_menciona_ciudad(addr: str, ciudad: str) -> bool:
    if not addr or not ciudad:
        return False
    return normalizar_ciudad(ciudad) in unidecode.unidecode(addr.lower())

# ======= GOOGLE PLACES =======
def search_nearby(lat, lng, name=None):
    """Busca sitios cerca de coords (radio 250m)"""
    global contador_llamadas
    url = "https://places.googleapis.com/v1/places:searchNearby"
    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": API_KEY,
        "X-Goog-FieldMask": "places.id,places.displayName,places.formattedAddress,places.location"
    }
    data = {
        "includedTypes": ["restaurant", "bar", "cafe"],
        "maxResultCount": 5,
        "languageCode": LANG,
        "locationRestriction": {
            "circle": {
                "center": {"latitude": lat, "longitude": lng},
                "radius": 250
            }
        }
    }
    r = requests.post(url, headers=headers, json=data)
    contador_llamadas += 1
    if r.status_code != 200:
        print(f"⚠️ Error nearby: {r.status_code} → {r.text}")
        return []
    return r.json().get("places", [])

def search_text_galicia(query: str):
    """Text Search limitado a Galicia"""
    global contador_llamadas
    url = "https://places.googleapis.com/v1/places:searchText"
    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": API_KEY,
        "X-Goog-FieldMask": "places.id,places.displayName,places.formattedAddress,places.location"
    }
    data = {
        "textQuery": query,
        "languageCode": LANG,
        "regionCode": "ES",
        "locationBias": {
            "rectangle": {
                "low": {"latitude": BOUNDING_BOX_GALICIA["min_lat"], "longitude": BOUNDING_BOX_GALICIA["min_lng"]},
                "high": {"latitude": BOUNDING_BOX_GALICIA["max_lat"], "longitude": BOUNDING_BOX_GALICIA["max_lng"]}
            }
        }
    }
    r = requests.post(url, headers=headers, json=data)
    contador_llamadas += 1
    if r.status_code != 200:
        print(f"⚠️ Error searchText: {r.status_code} → {r.text}")
        return []
    return r.json().get("places", []) or []

def obtener_detalles_place(place_id):
    global contador_llamadas
    url = f"https://places.googleapis.com/v1/places/{place_id}"
    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": API_KEY,
        "X-Goog-FieldMask": ",".join([
            "displayName","formattedAddress","regularOpeningHours","googleMapsUri",
            "rating","userRatingCount","priceLevel","location"
        ])
    }
    r = requests.get(url, headers=headers)
    contador_llamadas += 1
    if r.status_code != 200:
        return {}
    return r.json()

# ======= MATCHING =======
def elegir_mejor_candidato(nombre, ciudad, candidatos):
    """Combina nombre + ciudad + dirección + bbox"""
    n1 = normalizar_nombre(nombre or "")
    c1 = normalizar_ciudad(ciudad)

    mejor = None
    mejor_score = -1
    tipo = None

    for p in candidatos:
        dn = (p.get("displayName") or {}).get("text", "")
        dn_norm = normalizar_nombre(dn)
        addr = p.get("formattedAddress", "")

        # Validación por dirección y bbox
        loc = p.get("location") or {}
        lat, lng = loc.get("latitude"), loc.get("longitude")
        if not direccion_valida(addr) or not en_bbox_galicia(lat, lng):
            continue

        # Similaridad de nombre
        s = sim(dn_norm, n1)

        # Bonus si la dirección menciona la ciudad
        if c1 and direccion_menciona_ciudad(addr, ciudad):
            s += 0.1

        if s > mejor_score:
            mejor = p
            mejor_score = s
            if dn_norm == n1:
                tipo = "perfecto"
            elif s >= THRESHOLD_SIM_FUERTE:
                tipo = "fuerte"
            elif s >= THRESHOLD_SIM_MINIMO:
                tipo = "debil"

    return mejor, tipo

# ======= MAIN =======
with open(INPUT_FILE, "r", encoding="utf-8") as f:
    restaurantes = json.load(f)

actualizados, no_encontrados = [], []

for idx, rest in enumerate(restaurantes):
    if contador_llamadas >= MAX_LLAMADAS:
        break

    nombre = rest.get("nombre_final")
    ciudad = rest.get("ciudad")
    lat, lng = rest.get("latitude"), rest.get("longitude")

    print(f"[{idx+1}/{len(restaurantes)}] 🔎 {nombre} ({ciudad})")

    candidatos = []
    if lat and lng:
        candidatos = search_nearby(lat, lng, nombre)
    if not candidatos and nombre:
        q = f"{nombre} {ciudad}" if ciudad else nombre
        candidatos = search_text_galicia(q)

    if not candidatos:
        print(f"❌ No encontrado: {nombre}")
        no_encontrados.append(rest)
        continue

    mejor, tipo = elegir_mejor_candidato(nombre, ciudad, candidatos)
    if not mejor:
        print(f"❌ Sin buen candidato: {nombre}")
        no_encontrados.append(rest)
        continue

    detalles = obtener_detalles_place(mejor["id"])
    if not detalles:
        no_encontrados.append(rest)
        continue

    # Actualizar campos
    rest["name_google"] = (detalles.get("displayName") or {}).get("text", "")
    rest["address"] = detalles.get("formattedAddress", "")
    rest["google_maps_url"] = detalles.get("googleMapsUri")
    rest["open_hours_google"] = (detalles.get("regularOpeningHours") or {}).get("weekdayDescriptions")
    rest["rating_google"] = detalles.get("rating")
    rest["reviews_count_google"] = detalles.get("userRatingCount")
    rest["price_level_google"] = detalles.get("priceLevel")

    # Coordenadas solo si match perfecto
    if tipo == "perfecto":
        loc = detalles.get("location", {})
        rest["latitude"] = loc.get("latitude", rest.get("latitude"))
        rest["longitude"] = loc.get("longitude", rest.get("longitude"))

    actualizados.append(rest)
    time.sleep(0.2)

# ======= SALIDA =======
with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    json.dump(actualizados, f, ensure_ascii=False, indent=2)

with open(NO_ENCONTRADOS_FILE, "w", encoding="utf-8") as f:
    json.dump(no_encontrados, f, ensure_ascii=False, indent=2)

print(f"✅ Guardados {len(actualizados)} en {OUTPUT_FILE}")
print(f"❌ No encontrados: {len(no_encontrados)}")
print(f"ℹ️ Llamadas usadas: {contador_llamadas}")


ModuleNotFoundError: No module named 'unidecode'

### Validación de estados (abierto / cerrado)
Con el campo `businessStatus` de la API se detecta si un restaurante está:  
- ✅ Abierto (`OPERATIONAL`)  
- ⏳ Cerrado temporalmente (`CLOSED_TEMPORARILY`)  
- 🛑 Cerrado permanentemente (`CLOSED_PERMANENTLY`)  
Estos datos se guardan en `restaurantes_estado.json`.


In [ ]:
import requests
import json
import time
import difflib
import re
import unidecode
from urllib.parse import urlparse

# ======= CONFIG =======
API_KEY = ""  # ← pon tu clave aquí
INPUT_FILE = "restaurantes_combinados_actualizado_horarios.json"
OUTPUT_FILE = "restaurantes_estado.json"
NO_ENCONTRADOS_FILE = "no_encontrados_estado.json"

LANG = "es"
MAX_LLAMADAS = 50000
contador_llamadas = 0
MAX_QUERIES_PER_SITE = 8

# Bounding box de Galicia
BOUNDING_BOX_GALICIA = {
    "min_lat": 41.80,
    "max_lat": 43.85,
    "min_lng": -9.35,
    "max_lng": -6.50
}

THRESHOLD_SIM_FUERTE = 0.86
THRESHOLD_SIM_MINIMO = 0.40

# ======= UTILIDADES =======
STOPWORDS_NOMBRES = {
    "bar","restaurante","meson","mesón","parrillada","asador","taperia","tapería","pulperia","pulpería",
    "vinoteca","taberna","cerveceria","cervecería","cafeteria","cafetería","churrasqueria","churrasquería",
    "casa","de","comidas","o","el","la","los","las","del","dos","da","do","das","a","en","y","e"
}

def normalizar_texto(s: str) -> str:
    if not s:
        return ""
    s = unidecode.unidecode(s.lower())
    s = re.sub(r"[^a-z0-9\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def normalizar_nombre(nombre: str) -> str:
    s = normalizar_texto(nombre)
    tokens = [t for t in s.split() if t not in STOPWORDS_NOMBRES]
    return " ".join(tokens)

def sim(a: str, b: str) -> float:
    return difflib.SequenceMatcher(None, (a or ""), (b or "")).ratio()

def en_bbox_galicia(lat, lng) -> bool:
    if lat is None or lng is None:
        return False
    return (BOUNDING_BOX_GALICIA["min_lat"] <= lat <= BOUNDING_BOX_GALICIA["max_lat"] and
            BOUNDING_BOX_GALICIA["min_lng"] <= lng <= BOUNDING_BOX_GALICIA["max_lng"])

# --- Provincia desde TripAdvisor link ---
PROV_MAP = {
    "lugo": "Lugo",
    "pontevedra": "Pontevedra",
    "ourense": "Ourense",
    "orense": "Ourense",
    "a coruna": "A Coruña",
    "coruna": "A Coruña",
}
def extraer_provincia_de_link(link: str) -> str:
    if not link:
        return ""
    try:
        path = urlparse(link).path
        parts = [p.replace("_", " ").lower() for p in path.split("-") if p]
        for p in parts:
            for key, prov in PROV_MAP.items():
                if key in p:
                    return prov
    except:
        pass
    return ""

def provincia_aliases(prov: str):
    if not prov:
        return []
    base = unidecode.unidecode(prov).lower()
    if base == "a coruna":
        return ["a coruna", "coruna", "a coruña", "coruña"]
    if base == "ourense":
        return ["ourense", "orense"]
    return [prov, base]

def direccion_menciona_provincia(addr: str, prov: str) -> bool:
    if not addr or not prov:
        return False
    a = unidecode.unidecode(addr).lower()
    for alias in provincia_aliases(prov):
        if alias in a:
            return True
    return False

# ======= GOOGLE PLACES =======
def search_text_galicia(query: str):
    global contador_llamadas
    url = "https://places.googleapis.com/v1/places:searchText"
    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": API_KEY,
        "X-Goog-FieldMask": "places.id,places.displayName,places.location,places.formattedAddress"
    }
    data = {
        "textQuery": query,
        "languageCode": LANG,
        "regionCode": "ES",
        "locationBias": {
            "rectangle": {
                "low": {"latitude": BOUNDING_BOX_GALICIA["min_lat"], "longitude": BOUNDING_BOX_GALICIA["min_lng"]},
                "high": {"latitude": BOUNDING_BOX_GALICIA["max_lat"], "longitude": BOUNDING_BOX_GALICIA["max_lng"]}
            }
        }
    }
    r = requests.post(url, headers=headers, json=data)
    contador_llamadas += 1

    if r.status_code != 200:
        print(f"⚠️ Error searchText: {r.status_code} → {r.text}")
        return []

    resultados = r.json().get("places", []) or []
    filtrados = []
    for p in resultados:
        loc = p.get("location") or {}
        plat = loc.get("latitude")
        plng = loc.get("longitude")
        addr = p.get("formattedAddress")
        if en_bbox_galicia(plat, plng) or (addr and "galicia" in unidecode.unidecode(addr).lower()):
            filtrados.append(p)
    return filtrados

def obtener_detalles_place(place_id):
    """Solo pedimos el estado del negocio."""
    global contador_llamadas
    url = f"https://places.googleapis.com/v1/places/{place_id}"
    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": API_KEY,
        "X-Goog-FieldMask": "businessStatus"  # 👈 Solo este campo
    }
    r = requests.get(url, headers=headers)
    contador_llamadas += 1
    if r.status_code != 200:
        print(f"⚠️ Error en detalles: {r.status_code} → {r.text}")
        return {}
    return r.json()

# ======= MATCHING =======
def tokens(s: str) -> set:
    return set((s or "").split())

def elegir_mejor_candidato_por_nombre(nombre: str, candidatos):
    n1 = normalizar_nombre(nombre or "")
    mejor = None
    mejor_tipo = None
    mejor_score = -1.0

    for p in candidatos:
        dn = (p.get("displayName") or {}).get("text", "")
        dn_norm = normalizar_nombre(dn)

        if dn_norm and dn_norm == n1:
            return p, "perfecto"

        s = sim(dn_norm, n1)
        if s >= THRESHOLD_SIM_FUERTE and len(tokens(dn_norm) & tokens(n1)) >= 2:
            tipo = "fuerte"
        elif s >= THRESHOLD_SIM_MINIMO:
            tipo = "debil"
        else:
            tipo = None

        if tipo and s > mejor_score:
            mejor = p
            mejor_tipo = tipo
            mejor_score = s

    return mejor, mejor_tipo

# ======= EJECUCIÓN =======
with open(INPUT_FILE, "r", encoding="utf-8") as f:
    restaurantes = json.load(f)

resultado = []
no_encontrados = []

for idx, rest in enumerate(restaurantes):
    if contador_llamadas >= MAX_LLAMADAS:
        print("⚠️ Límite de llamadas alcanzado. Deteniendo proceso.")
        break

    nombre_original = (rest.get("name") or "").strip()
    link_trip = (rest.get("link") or "").strip()
    provincia_hint = extraer_provincia_de_link(link_trip)
    base_norm = normalizar_nombre(nombre_original)

    print(f"[{idx+1}/{len(restaurantes)}] 🔎 Buscando: '{nombre_original}'"
          + (f" | prov: {provincia_hint}" if provincia_hint else ""))

    queries = []
    if nombre_original:
        queries.append(nombre_original)
    if base_norm and base_norm != nombre_original:
        queries.append(base_norm)

    prefijos = ["Restaurante", "Mesón", "Parrillada", "Bar"]
    for pref in prefijos:
        if nombre_original:
            q = f"{pref} {nombre_original}"
            if q not in queries:
                queries.append(q)

    if provincia_hint:
        extra = [f"{nombre_original} {provincia_hint}"]
        if base_norm and base_norm != nombre_original:
            extra.append(f"{base_norm} {provincia_hint}")
        for pref in ["Restaurante", "Mesón", "Parrillada", "Bar"]:
            extra.append(f"{pref} {nombre_original} {provincia_hint}")
        for q in extra:
            if q not in queries:
                queries.append(q)

    if MAX_QUERIES_PER_SITE and len(queries) > MAX_QUERIES_PER_SITE:
        queries = queries[:MAX_QUERIES_PER_SITE]

    candidatos_por_id = {}
    match_perfecto_encontrado = None

    for q in queries:
        if not q:
            continue
        cand = search_text_galicia(q)
        if provincia_hint:
            cand_prov = [p for p in cand if direccion_menciona_provincia(p.get("formattedAddress",""), provincia_hint)]
            if cand_prov:
                cand = cand_prov

        for p in cand:
            pid = p.get("id")
            if pid and pid not in candidatos_por_id:
                candidatos_por_id[pid] = p

        for p in cand:
            dn = (p.get("displayName") or {}).get("text", "")
            if normalizar_nombre(dn) == base_norm:
                match_perfecto_encontrado = p
                break

        if match_perfecto_encontrado:
            break

        time.sleep(0.2)

    if match_perfecto_encontrado:
        mejor = match_perfecto_encontrado
        tipo_match = "perfecto"
    else:
        candidatos = list(candidatos_por_id.values())
        if not candidatos:
            print("❌ No encontrados")
            no_encontrados.append(rest)
            continue
        mejor, tipo_match = elegir_mejor_candidato_por_nombre(nombre_original, candidatos)
        if not mejor:
            print("❌ Sin match suficiente")
            no_encontrados.append(rest)
            continue

    # Obtener solo businessStatus
    place_id = mejor.get("id")
    detalles = obtener_detalles_place(place_id) if place_id else {}
    status = detalles.get("businessStatus")
    rest["business_status"] = status

    if status == "CLOSED_PERMANENTLY":
        print("   🛑 Cerrado permanentemente")
    elif status == "CLOSED_TEMPORARILY":
        print("   ⏳ Cerrado temporalmente")
    elif status == "OPERATIONAL":
        print("   ✅ Abierto")
    else:
        print("   ❓ Estado desconocido")

    resultado.append(rest)

    if (idx + 1) % 100 == 0:
        print(f"📊 Progreso: {idx+1} procesados — {contador_llamadas} llamadas usadas")

# ======= SALIDA =======
with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    json.dump(resultado, f, ensure_ascii=False, indent=4)

with open(NO_ENCONTRADOS_FILE, "w", encoding="utf-8") as f:
    json.dump(no_encontrados, f, ensure_ascii=False, indent=4)

print(f"\n✅ Guardado '{OUTPUT_FILE}' con {len(resultado)} restaurantes.")
print(f"❗ Guardado '{NO_ENCONTRADOS_FILE}' con {len(no_encontrados)} no encontrados.")
print(f"ℹ️ Llamadas usadas: {contador_llamadas}")


Se han truncado las últimas 5000 líneas del flujo de salida.
   ✅ Abierto
[1484/3958] 🔎 Buscando: 'Avenida' | prov: A Coruña
   ✅ Abierto
[1485/3958] 🔎 Buscando: 'O Cazolo Furado SL.' | prov: A Coruña
   ✅ Abierto
[1486/3958] 🔎 Buscando: 'Javier' | prov: A Coruña
   🛑 Cerrado permanentemente
[1487/3958] 🔎 Buscando: 'Torre Internacional' | prov: A Coruña
   ✅ Abierto
[1488/3958] 🔎 Buscando: 'Meson Paredes' | prov: A Coruña
   ❓ Estado desconocido
[1489/3958] 🔎 Buscando: 'Piccaben' | prov: A Coruña
   🛑 Cerrado permanentemente
[1490/3958] 🔎 Buscando: 'Cafe Bar Mediodia' | prov: A Coruña
   ✅ Abierto
[1491/3958] 🔎 Buscando: 'McDonald's' | prov: A Coruña
   ✅ Abierto
[1492/3958] 🔎 Buscando: 'Montecarlo Casa de Vinos' | prov: A Coruña
   ✅ Abierto
[1493/3958] 🔎 Buscando: 'Sabores' | prov: A Coruña
   ✅ Abierto
[1494/3958] 🔎 Buscando: 'Cerveceria Sidreria Rosalia' | prov: A Coruña
   ✅ Abierto
[1495/3958] 🔎 Buscando: 'Pizzeria A Quintana' | prov: A Coruña
   🛑 Cerrado permanentemente
[1496/3

In [ ]:
from google.colab import files
import json

# Guardar en local con los nombres que quieras
with open("restaurantes_estado.json", "w", encoding="utf-8") as f:
    json.dump(resultado, f, ensure_ascii=False, indent=4)

with open("no_encontrados_estado.json", "w", encoding="utf-8") as f:
    json.dump(no_encontrados, f, ensure_ascii=False, indent=4)

# Descargar los archivos a tu máquina
files.download("restaurantes_estado.json")
files.download("no_encontrados_estado.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import json
import pandas as pd

# --- Cargar el archivo ---
FILE = "restaurantes_estado.json"

with open(FILE, "r", encoding="utf-8") as f:
    data = json.load(f)

print(f"✅ Total de registros: {len(data)}")

# --- Mostrar primeras filas como tabla ---
df = pd.DataFrame(data)

# Muestra las primeras 5 filas (puedes cambiar el número)
display(df.head())


✅ Total de registros: 3958


,id,name,link,rating,reviews,price_range_usd,latitude,longitude,cuisines,is_sponsored,...,delivery_url,price_range,diets,meal_types,dining_options,owner_types,top_tags,detalle_completo,google_maps_url,name_google
0,15052579,Espolada,https://www.tripadvisor.com/Restaurant_Review-...,4.0,5,None,43.012450,-7.551620,[Spanish],False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,992437,O Dezaseis,https://www.tripadvisor.com/Restaurant_Review-...,4.2,1963,NaN,42.881725,-8.539337,"[European, Spanish, Healthy]",NaN,...,None,$$ - $$$,"[Vegetarian friendly, Vegan options, Gluten fr...","[Lunch, Dinner, Late Night]","[Reservations, Seating, Serves Alcohol, Full B...",[],"[Mid-range, European, Spanish, Vegetarian frie...",NaN,NaN,NaN
2,10289534,Parrilla O Establo,https://www.tripadvisor.com/Restaurant_Review-...,4.3,80,NaN,42.873608,-8.595798,"[Steakhouse, Barbecue]",NaN,...,None,$$ - $$$,[],"[Lunch, Dinner]","[Reservations, Outdoor Seating, Seating, Wheel...",[],"[Mid-range, Steakhouse, Barbecue]",NaN,NaN,NaN
3,10173814,Bar Gaiola,https://www.tripadvisor.com/Restaurant_Review-...,3.7,123,$,42.877316,-8.545388,[Spanish],False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN
4,13403422,Lasso. Cafeteria do Auditorio de Galicia,https://www.tripadvisor.com/Restaurant_Review-...,4.5,2,NaN,42.889748,-8.544891,"[European, Spanish, Pub, Gastropub]",NaN,...,None,$$ - $$$,[],"[Breakfast, Lunch, Drinks]","[Takeout, Buffet, Seating, Parking Available, ...",[],"[Mid-range, European, Spanish]",NaN,NaN,NaN
